## Week 3 - Edge and Line Detection 

In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import math

dataDir = './data'

6. Edge detection - Sobel filter

In [2]:
# 6 a) Calculate the first derivatives of the image in the x and y direction, using the Sobel function

# Load the image

"""
" @param kernelSize The size of the kernel to use for the Sobel filter
"""
def sobelFilter(imageSrc, kernelSize = 3, useGaussianBlur = False, gaussianBlurKernelSize = 3):
    img = cv2.imread(imageSrc)
    # Check if image was loaded correctly
    if img is None:
        print("error opening image" + imageSrc)
    else:
        # Can apply a gaussian blur to reduce noise
        if useGaussianBlur:
            img = cv2.GaussianBlur(img, (gaussianBlurKernelSize, gaussianBlurKernelSize), 0)

        # Convert to grayscale
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Define variables for the Sobel function
        ddepth = cv2.CV_16S
        scale = 1
        delta = 0

        # Calculate the first derivate in the x direction
        grad_x = cv2.Sobel(gray_img, ddepth, 1, 0, ksize= kernelSize, scale=scale, delta=delta, borderType=cv2.BORDER_DEFAULT)

        # Calculate the first derivate in the y direction
        grad_y = cv2.Sobel(gray_img, ddepth, 0, 1, ksize= kernelSize, scale=scale, delta=delta, borderType=cv2.BORDER_DEFAULT)
        # Could also try the cv2.Scharr function to approximate the derivatives

        # print("grad.x.shape = " + str(grad_x.shape))
        # print("grad.y.shape = " + str(grad_y.shape))
        # print("grad_x" + str(grad_x))
        # print("grad_y" + str(grad_y))

        return grad_x, grad_y


imageSrc = os.path.join(dataDir, 'images/FEUP_01.jpg')

# Call the method and get the gradients for each axis
grad_x, grad_y = sobelFilter(imageSrc, 3)

In [3]:
# 6b) Calculate the approximate value of the gradient by combining the directional directives

def combineSobelGradients(grad_x, grad_y, weight1 = 0.5, weight2 = 0.5):
    # Calculate the gradient magnitude in the x and y direction
    abs_grad_x = cv2.convertScaleAbs(grad_x)
    abs_grad_y = cv2.convertScaleAbs(grad_y)

    # Combine the directional derivatives
    grad = cv2.addWeighted(abs_grad_x, 0.5, abs_grad_y, 0.5, 0)

    # print the combined gradient
    # print("grad.shape" + str(grad.shape))
    # print("grad" + str(grad))
    return grad

# Call the method to combine the gradients
combinedGrad = combineSobelGradients(grad_x, grad_y, 0.5, 0.5)

In [4]:
# 6c) Show the gradient image

cv2.imshow("Sobel derivatives image", combinedGrad)

# close the window
cv2.waitKey(0)
cv2.destroyAllWindows()

In [5]:
# Show the result of thresholding the gradient image; use a trackbar to select the threshold value

MAX_VALUE = 255

def showSobelWithThreshold(grad, window_name = "Thresholded gradient image"):
    # Define a function to callback when the trackbar is changed
    def on_trackbar(new_val):
        # Threshold the gradient image
        _ret, thresholdedGradImg = cv2.threshold(grad, new_val, MAX_VALUE, cv2.THRESH_BINARY)
        cv2.imshow(window_name, thresholdedGradImg)

    # Define the window
    cv2.namedWindow(window_name)

    # Create the trackbar
    trackbar_name = "Threshold Trackbar"
    cv2.createTrackbar(trackbar_name, window_name, 0, MAX_VALUE, on_trackbar)

    # Show the initial image
    cv2.imshow(window_name, grad)
        
    # Close the window
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Call the method to show the thresholded gradient image
showSobelWithThreshold(combinedGrad)

In [ ]:
# 6e) try different kernel sizes
for i in range(1, 8, 2):
    grad_x, grad_y = sobelFilter(imageSrc, i)
    combinedGrad = combineSobelGradients(grad_x, grad_y, 0.5, 0.5)

    # Show the gradient image with threshold trackbar and current kernel size
    windowName = "Sobel derivatives image with kernel size " + str(i)
    showSobelWithThreshold(combinedGrad, windowName)

    # close the window
    cv2.waitKey(0)
    cv2.destroyAllWindows()

grad_x[[ 0  0 -1 ... 10 10  0]
 [ 0  1  0 ... 15 18  0]
 [ 0  0  1 ... 15 20  0]
 ...
 [ 0 -8 -1 ...  0  0  0]
 [ 0 -2 -3 ...  1  0  0]
 [ 0  2 -4 ... -3  0  0]]
grad_y[[ 0  0  0 ...  0  0  0]
 [ 1  1  1 ...  0  4 10]
 [ 1  0  0 ...  3  3  0]
 ...
 [ 4  7  9 ...  0  0 -1]
 [ 1  8 11 ... -1  0 -1]
 [ 0  0  0 ...  0  0  0]]
grad.shape(787, 1181)
grad[[ 0  0  0 ...  5  5  0]
 [ 0  1  0 ...  8 11  5]
 [ 0  0  0 ...  9 12  0]
 ...
 [ 2  8  5 ...  0  0  0]
 [ 0  5  7 ...  1  0  0]
 [ 0  1  2 ...  2  0  0]]
grad_x[[  0   2  -2 ...  50  56   0]
 [  0   2   0 ...  55  66   0]
 [  0   1   2 ...  59  73   0]
 ...
 [  0 -25  -6 ...   1   1   0]
 [  0 -10 -11 ...  -1   0   0]
 [  0   0 -14 ...  -4   0   0]]
grad_y[[ 0  0  0 ...  0  0  0]
 [ 4  4  6 ...  3 18 28]
 [ 2  1  0 ... 13  9  6]
 ...
 [22 27 30 ... -1 -1 -2]
 [18 28 35 ...  1 -2 -2]
 [ 0  0  0 ...  0  0  0]]
grad.shape(787, 1181)
grad[[ 0  1  1 ... 25 28  0]
 [ 2  3  3 ... 29 42 14]
 [ 1  1  1 ... 36 41  3]
 ...
 [11 26 18 ...  1  1  1]
 [ 

In [6]:
# 6f) Test the effect of applying a Gaussian blur before applying the Sobel filter, use gaussian filters with increasing sizes 
# (ex. 3x3, 7x7, 11x11, 31x31)
for kernelSize in range(1, 8, 2):
    for gaussFilterSize in [3, 7, 11, 31]:
        grad_x, grad_y = sobelFilter(imageSrc, kernelSize, True, gaussFilterSize)
        combinedGrad = combineSobelGradients(grad_x, grad_y, 0.5, 0.5)

        # Show the gradient image with threshold trackbar and current kernel size
        windowName = f"Sobel derivatives image with kernel size {str(kernelSize)} and gaussFilterSize {str(gaussFilterSize)}" 
        showSobelWithThreshold(combinedGrad, windowName)

        # close the window
        cv2.waitKey(0)
        cv2.destroyAllWindows()

7. Edge Detection - Canny filter

In [7]:
# 7a) Detect the edges of an image using the Canny openCV function. use trackbars to select different
# low and high threshold for the hysteresis procedure and different aperture size for the Sobel() function.
MAX_VALUE_HYSTERESIS = 255
MAX_VALUE_APERTURE = 2  # 0 -> 3, 1-> 5, 2 -> 7

def CannyWithThreshold(imageSrc, window_name = "Canny image"):
    # load the image
    img = cv2.imread(imageSrc)
    # Check if image was loaded correctly
    if img is None:
        print("error opening image" + imageSrc)
        return
    
    # Define a function to callback when the trackbar is changed
    def on_trackbar_hysteresis_high(new_val):
        # Threshold the gradient image
        highThreshold = new_val
        cannyImg = cv2.Canny(img, lowThreshold, highThreshold, apertureSize)
        cv2.imshow(window_name, cannyImg)

    def on_trackbar_hysteresis_low(new_val):
        # Threshold the gradient image
        lowThreshold = new_val
        cannyImg = cv2.Canny(img, lowThreshold, highThreshold, apertureSize)
        cv2.imshow(window_name, cannyImg)

    def on_trackbar_aperture(new_val):
        # Threshold the gradient image
        apertureSize = 2*(new_val+1) + 1
        cannyImg = cv2.Canny(img, lowThreshold, highThreshold, apertureSize)
        cv2.imshow(window_name, cannyImg)
    
    # Define the window
    cv2.namedWindow(window_name)

    # Create the trackbar
    trackbar_name = "Threshold Trackbar"
    lowThreshold = 0
    highThreshold = MAX_VALUE_HYSTERESIS
    apertureSize = 3    # Possible values are 3, 5, and 7. apertureSize = 2*(newValue+1) + 1
    # Create trackbars for hysteresis thresholds
    cv2.createTrackbar(f"{trackbar_name}_high", window_name, highThreshold, MAX_VALUE, on_trackbar_hysteresis_high)
    cv2.createTrackbar(f"{trackbar_name}_low", window_name, lowThreshold, MAX_VALUE, on_trackbar_hysteresis_low)
    # Create trackbar for aperture size
    cv2.createTrackbar(f"{trackbar_name}_aperture", window_name, 0, MAX_VALUE_APERTURE, on_trackbar_aperture)

    # Apply the initial Canny filter
    cannyImg = cv2.Canny(img, lowThreshold, highThreshold, apertureSize)

    # Show the initial image
    cv2.imshow(window_name, cannyImg)

    # Close the window
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Call the method to show the thresholded gradient image
imageSrc = os.path.join(dataDir, 'images/FEUP_01.jpg')
CannyWithThreshold(imageSrc)



In [8]:
# 7b) Compare the results of applying the following 2 filters to the same image:
# 1) Sobel filter, with threshold t, after smoothing the image with a Gaussian blur filter with size s;
# 2) Canny filter, with "low threshold" = "high threshold" = t and "aperture" = s,
#    using the same t and s values. Try also with a "low threshold" different from the "high threshold".

# Common parameters
imageSrc = os.path.join(dataDir, 'images/FEUP_01.jpg')
MAX_VALUE = 255
t = 100
s = 3

# Sobel filter
# ===============================================================
# Call the method and get the gradients for each axis
grad_x, grad_y = sobelFilter(imageSrc, s)
# Combine the gradients
combinedGrad = combineSobelGradients(grad_x, grad_y, 0.5, 0.5)
# Threshold the gradient image
_ret, thresholdedGradImg = cv2.threshold(combinedGrad, t, MAX_VALUE, cv2.THRESH_BINARY)
# ===============================================================

# Canny filter
# ===============================================================
# Canny Img with high and low threshold equal
cannyImg = cv2.Canny(cv2.imread(imageSrc), t, t, apertureSize=s)
# Canny Img with high and low threshold different
cannyImg2 = cv2.Canny(cv2.imread(imageSrc), max(0, t/2), min(t*2, MAX_VALUE), apertureSize=s)


# Resize images to fit the screen
thresholdedGradImg = cv2.resize(thresholdedGradImg, (0,0), fx=0.7, fy=0.7)
cannyImg = cv2.resize(cannyImg, (0,0), fx=0.7, fy=0.7)
cannyImg2 = cv2.resize(cannyImg2, (0,0), fx=0.7, fy=0.7)
# ===============================================================

# Show both images
cv2.imshow('Sobel filter', thresholdedGradImg)
cv2.imshow('Canny filter', cannyImg)
cv2.imshow('Canny filter w/ Low != High Threshold', cannyImg2)

# close the window
cv2.waitKey(0)
cv2.destroyAllWindows()


8. Hough Transform - Line and Circle Detection

In [9]:
# 8a) Compare the functionality of HoughLines() and HoughLinesP() OpenCV functions for line detection
# The HoughLines() function applies the Hough Transform to detect lines in an image and returns an array of (rho, theta) pairs representing the detected lines.

# The HoughLinesP() function is a Probabilistic Hough Transform. It's an optimization of the Hough Transform that only takes a sample of the points in the image to detect lines,
# which is sufficient for line detection. It is used for time efficiency.

In [63]:
# 8b) Use HoughLines() to detect lines in images like those in figure 1.a and 1.b; try different parameter values; draw
# the detected lines on the image. 

def showHoughLines(src, cannyLowThresh, cannyHighThresh, cannyApertureSize, houghRho, houghTheta, houghThreshold, srn=0, stn=0, windowName = "Hough Lines"):
    # Load the image
    img = cv2.imread(src)
    # Check if image was loaded correctly
    if img is None:
        print("error opening image" + src)
        exit()

    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Detect the edges of the input image, with a Canny Filter
    cannyEdges = cv2.Canny(gray, cannyLowThresh, cannyHighThresh, apertureSize=cannyApertureSize)

    # Copy edges to the image that will display the results in RGB (because of matplotlib)
    # Remember that OpenCV operates with BGR
    cannyImgColored = cv2.cvtColor(cannyEdges, cv2.COLOR_GRAY2RGB)

    # Apply the Hough Transform to detect lines
    lines = cv2.HoughLines(cannyEdges, houghRho, houghTheta, houghThreshold, srn=srn, stn=stn)

    # Draw the detected lines on the image
    if lines is not None:
        for i in range(0, len(lines)):
            # print("line", i, ": rho =", lines[i][0][0], "theta =", lines[i][0][1])
            rho = lines[i][0][0]
            theta = lines[i][0][1]
            a = math.cos(theta)
            b = math.sin(theta)
            x0 = a*rho
            y0 = b*rho
            pt1 = (int(x0 + 1000*(-b)), int(y0 + 1000*(a)))
            pt2 = (int(x0 - 1000*(-b)), int(y0 - 1000*(a)))
            lineThickness = 2
            cv2.line(cannyImgColored, pt1, pt2, (0, 0, 255), lineThickness, cv2.LINE_AA)

    # Show the image
    cv2.imshow(windowName, cannyImgColored)

    # Close the window
    cv2.waitKey(0)
    cv2.destroyAllWindows()


imageSrc1 = os.path.join(dataDir, 'images/chessboard_02.jpg')
imageSrc2 = os.path.join(dataDir, 'images/streetLines_01.jpg')
imageSrc3 = os.path.join(dataDir, 'images/coins_02.jpg')

cannyLowThresh = 50 # Canny low threshold
cannyHighThresh = 180   # Canny high threshold
cannyApertureSize = 3   # Canny aperture size
houghRho = 1    # Distance resolution of the accumulator in pixels
houghTheta = np.pi/180  # Angle resolution of the accumulator in radians
houghThreshold = 200    # Accumulator threshold parameter. Only those lines are returned that get enough votes (> threshold)

# Show the results
# showHoughLines(imageSrc, cannyLowThresh, cannyHighThresh, cannyApertureSize, houghRho, houghTheta, houghThreshold, windowName="Hough Lines Chessboard")

# showHoughLines(imageSrc2, 50, 200, 3, 1, np.pi/180, 150, windowName="HoughLinesStreet")

showHoughLines(imageSrc3, cannyLowThresh, cannyHighThresh, cannyApertureSize, 1, np.pi/180, 180, windowName="HoughlinesCoins")

# Changing the houghThreshold parameter results in different lines being detected.

In [133]:
# 8c) Use HoughLinesP() to detect line segments in the same images that you used in the previous problem; try
# different parameter values; draw the detected line segments on the image.

def showHoughLinesP(src, cannyLowThresh, cannyHighThresh, cannyApertureSize, houghRho, houghTheta, houghThreshold, minLineLength=0, maxLineGap=0, windowName = "Hough Lines"):
    # Load the image
    img = cv2.imread(src)
    # Check if image was loaded correctly
    if img is None:
        print("error opening image" + src)
        exit()

    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Detect the edges of the input image, with a Canny Filter
    cannyEdges = cv2.Canny(gray, cannyLowThresh, cannyHighThresh, apertureSize=cannyApertureSize)

    # Copy edges to the image that will display the results in RGB (because of matplotlib)
    # Remember that OpenCV operates with BGR
    cannyImgColored = cv2.cvtColor(cannyEdges, cv2.COLOR_GRAY2RGB)

    # Apply the Hough Transform to detect lines
    lines = cv2.HoughLinesP(cannyEdges, houghRho, houghTheta, houghThreshold, minLineLength=minLineLength, maxLineGap=maxLineGap)

    # Draw the detected lines on the image
    if lines is not None:
        for i in range(0, len(lines)):
            l = lines[i][0]
            lineThickness = 2
            cv2.line(cannyImgColored, (l[0], l[1]), (l[2], l[3]), (255,0,0), lineThickness, cv2.LINE_AA)

    # Show the image
    cv2.imshow(windowName, cannyImgColored)

    # Close the window
    cv2.waitKey(0)
    cv2.destroyAllWindows()


imageSrc1 = os.path.join(dataDir, 'images/chessboard_02.jpg')
imageSrc2 = os.path.join(dataDir, 'images/streetLines_01.jpg')
imageSrc3 = os.path.join(dataDir, 'images/coins_02.jpg')

# Show the results
# showHoughLinesP(imageSrc, 50, 180, 3, 1, np.pi/180, 60, minLineLength=0, maxLineGap=60, windowName="Hough Lines Chessboard")

# showHoughLinesP(imageSrc2, 50, 200, 3, 1, np.pi/180, 150, minLineLength=200, maxLineGap=200, windowName="HoughLinesStreet")

showHoughLinesP(imageSrc3, 50, 180, 3, 1, np.pi/180, 180, minLineLength=1, maxLineGap=10, windowName="HoughlinesCoins")

# Changing the houghThreshold parameter results in different lines being detected.

In [17]:
# 8d) Use HoughCircles() to detect the coins present in images like those in figure 1.c and 1.d (without or with
# superposition among the coins).

# param 1 -> higher threshold of the two passed to the Canny edge detector
# param 2 -> accumulator threshold for the circle centers at the detection stage
# minRadius -> minimum circle radius (if < 0, uses 0)
# maxRadius -> maximum circle radius. If <= 0, uses the maximum image dimension. If < 0, returns centers without finding the radius.
def showHoughCircles(src, param1, param2, minRadius, maxRadius):
    img = cv2.imread(src) # Change this, according to your image's path

    # Convert to grayscale
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    gray_img = cv2.blur(gray_img, (3,3))

    # Copy edges to the image that will display the results
    imgCopy = cv2.cvtColor(gray_img, cv2.COLOR_GRAY2RGB)

    # Show the original image
    cv2.imshow('Original Image', img)

    # Apply the Hough circle transform
    detectionMethod = cv2.HOUGH_GRADIENT # corresponds to the canny filter
    resolutionFlag = 1 # same resolution as the original image
    minDistance = 20 # between the centers of the detected circles

    # param1 and param2 are the thresholds passed to the detection method 
    circles = cv2.HoughCircles(gray_img, detectionMethod, resolutionFlag, minDistance, param1=param1, param2=param2, minRadius=minRadius, maxRadius=maxRadius)
    circles = np.uint16(np.around(circles))

    # Drawing the resulting circles
    for i in circles[0,:]: 
        cv2.circle(imgCopy, (i[0],i[1]), i[2], (0,255,0), 2)

    cv2.imshow('Hough Circle Transform Result', imgCopy)

    # Close the window
    cv2.waitKey(0)
    cv2.destroyAllWindows()

imageSrc1 = os.path.join(dataDir, 'images/coins_01.jpg')
imageSrc2 = os.path.join(dataDir, 'images/coins_02.jpg')

# Show the results
# showHoughCircles(imageSrc1, 210, 50, 0, 0)

showHoughCircles(imageSrc2, 550, 50, 0, 0)  # Why does the high threshold have to be so high? Shouldn't the max be 255?